# 20200925_ノンプロTA用　PDF→画像変換ツール

In [123]:
import pathlib
import re
from io import StringIO

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import pdf2image


def get_text_from_pdf():
    """PDFファイルから演習問題に該当するテキストをページ数と共に抽出する関数
    Returns:
        dict: 演習問題一覧。ページ数をキーとして、演習ナンバーと本文が入った辞書
    """
    # pdfminerの設定
    rsrcmgr = PDFResourceManager()
    codec = 'utf-8'
    laparams = LAParams()
    laparams.detext_vertical=True

    search_text = r'演習\s*\d－\d+'  # 抽出する文字列(正規表現：ここでは「演習○－○」)
    ensyu_dict = {}

    with open(PDF_FILEPATH, 'rb') as fp:
        for i, page in enumerate(PDFPage.get_pages(fp)):
            outfp = StringIO()
            device = TextConverter(
                rsrcmgr,
                outfp,
                codec=codec,
                laparams=laparams
                )
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            interpreter.process_page(page)

            output_text = outfp.getvalue() \
                .replace('\u2028', '') \
                .split('Copyright')[0] \
                .rstrip()

            output_page = re.search(search_text, output_text)  # 演習問題ページならTrue

            # 演習問題リスト作成
            if output_page:
                ensyu_dict[i+1] = (output_page.group(), output_text)

    return ensyu_dict


def save_text(ensyu_dict):
    """演習問題一覧をテキストファイル出力
    Args1:
        ensyu_dict(dict): 演習問題一覧。ページ数をキーとして、演習ナンバーと本文が入った辞書
    """
    output_text_path = OUTPUT_DIRPATH / f'0{COURCE_NO}_ensyu.txt'
    with output_text_path.open('w') as f:
        print(ensyu_dict, file=f)


def save_images(ensyu_dict):
    """演習問題を画像ファイル出力
    Args1:
        ensyu_dict(dict): 演習問題一覧。ページ数をキーとして、演習ナンバーと本文が入った辞書
    """
    # - convert_from_path関数には1ページごとの画像のリストが入る
    # - mkidr(exist_ok=True)にすると上書きされるっぽい
    img_dirpath = OUTPUT_DIRPATH / f'0{COURCE_NO}'
    pathlib.Path(img_dirpath).mkdir(exist_ok=True)

    images = pdf2image.convert_from_path(PDF_FILEPATH, size=1280)
    for i, image in enumerate(images):
        if i+1 in ensyu_dict:
            filename = ensyu_dict[i+1][0].replace(' ', '')
            image_filepath = img_dirpath / f'{filename}.png'
            image.save(image_filepath)


# グローバル変数。準備したい講座回数・資料PDFファイルのパス・画像出力先フォルダのパスを記入
COURCE_NO = 2
COURCE_DATE = [20200923, 20200930,]
PDF_FILEPATH = pathlib.Path(
    f'/Users/m.ohsaki/Dropbox/○MyDataBox/ノンプロ研/{COURCE_DATE[COURCE_NO-1]}初心者講座Pythonコース-0{COURCE_NO}.pdf'
    )
OUTPUT_DIRPATH = pathlib.Path(
    '/Users/m.ohsaki/Dropbox/○MyDataBox/ノンプロ研/Python初級講座4期'
    )

# 関数実行
ensyu_dict = get_text_from_pdf()
save_text(ensyu_dict)
save_images(ensyu_dict)